In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
df = pd.read_csv('amazon_products.csv')
df.drop(columns=['productURL','listPrice','boughtInLastMonth'], inplace=True)
df = df.dropna(subset=['price', 'category_id'], how='all')

PLOTTING CATEGORIES WITH MOST PRODUCTS

In [ ]:
plt.subplots(figsize=(12,10))
list1 = []
for i in df['category_id']:
    list1.append(str(i))
ax = pd.Series(list1).value_counts()[:15].sort_values(ascending=True).plot.barh(width=0.9,color=sns.color_palette('hls',10))
for i, v in enumerate(pd.Series(list1).value_counts()[:15].sort_values(ascending=True).values):
    ax.text(.8, i, v,fontsize=12,color='white',weight='bold')
plt.title('-Top 15 Categories-')
plt.ylabel('Category ID')
plt.xlabel('Number of Products')

plt.show()

ONE HOT ENCODING THE CATEGORIES


In [ ]:
encoder = OneHotEncoder(categories=[list(range(1, 271))],sparse_output=False)  # Use sparse_output=False to get a DataFrame
encoded_array = encoder.fit_transform(df[['category_id']])
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(['category_id']))

df = pd.concat([df, encoded_df], axis=1)

# Drop original categorical columns (optional)
df.drop(['category_id'], axis=1, inplace=True)

DATA CLEANING AND FEATURE SELECTION



In [ ]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df = df[df['category_id_134'] != 1.0]

features = data.drop(['asin', 'imgUrl', 'title'], axis=1)
features.iloc[:, 4:] *= 10  # Multiply all categorical columns increasing weight of category

TRAINING THE KNN MODEL


In [ ]:
knn = NearestNeighbors(n_neighbors=15, metric='cosine', n_jobs=-1)
knn.fit(features)

SAVING THE MODEL

In [ ]:
joblib.dump(knn, 'models/knn_model.pkl', compress=3)
joblib.dump(data, 'models/original_data.pkl', compress=3)